# **Cleaning the Dataset :**


A lot of Inconsistency was found when I tried to manually analise the data. Some column's data were interchanged while 
other columns had false data, which I have cleaned manually.

The Date column in new patients data provided was not in correct format. Thus I had to format it manually.

In order to have a have a better and stronger dataset, I have cleaned it in the following way:



In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import openpyxl
from openpyxl import load_workbook
import matplotlib.pyplot as plt
from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
excel_linelist = pd.read_excel(r'/content/gdrive/MyDrive/IITB - INTERNSHIP/Dataset/Provided Data/MASTER_CLOSED_LINELIST_JV.xlsx', sheet_name=None) # Importing the excel sheet
combined = pd.concat(excel_linelist.values())     # Combining worksheets
# Linelist
df_linelist = pd.DataFrame(data=combined)    # Creating pd Dataframe
# New Patients list
df_linelist2 = pd.read_excel(r'/content/gdrive/MyDrive/IITB - INTERNSHIP/Dataset/Provided Data/new patients list.xlsx', sheet_name='New Patient list') # Importing the excel sheet
# JV Contact list
df_linelist3 = pd.read_excel(r'/content/gdrive/MyDrive/IITB - INTERNSHIP/Dataset/Provided Data/JV-CONTACT TRACE.xlsx') # Importing the excel sheet

# Date in wrong format new patients list
for i in range(0,len(df_linelist2)):
  x = str(df_linelist2.loc[i]['Date']).split(".")
  df_linelist2.loc[i,'Date'] = datetime(int(x[2]), int(x[1]), int(x[0]))

#combining all df
frames = [df_linelist, df_linelist2, df_linelist3]
df = pd.concat(frames)
df['Date'] = pd.to_datetime(df['Date'])


df_ambulance = pd.read_excel(r'/content/gdrive/MyDrive/IITB - INTERNSHIP/Dataset/Provided Data/Ambulance_Report.xlsx') # Importing the excel sheet

## Data Preprocessing:
---
1. Dropping/removing duplicate rows.
2. Sorting the data according to the date.
3. Removing/dropping all empty columns.
4. Dropping the unwanted columns:

        i.    S No
        ii.   Ward -->  As the dataset is from the S ward, we don't need to soecify it and increase the dataset.
        iii.  Person / Official from other ward --> This won't help us in determining the death/recovery rate.
        iv.   Locality check  
        v.    Comments / Remarks
        vi.   Remarks from Ward
        vii.  Calls check?
        viii. Reasons why case is outside CZ, SB
        ix.   Action Needed from Ward --> Almost Empty


5. Removing 'Home Isolation' in 'DCH / DCHC / CCC2 name' column, as it is not needed there.


In [ ]:
# Cleaning linelist, New Patients list, JV Contact list
df_dup = df.drop_duplicates(subset = 'Test ID (ICMR)')  # dropping the duplicates
df_sort = df_dup.sort_values(by=['Age'])   # sorting the complete dataset by the dates
df_empt = df_sort.dropna(how='all', axis=1)   # dropping all the empty columns
df_new = df_empt.drop(columns = ['S No','Ward','Person / Official from other ward', 'Locality check', 'Comments / Remarks', 'Remarks from Ward','Calls check?', 'Reasons why case is outside CZ, SB', 'Action Needed from Ward', 'Select Symptoms'])
#df_new['DCH / DCHC / CCC2 name'].replace('Home Isolation', ' ') # Removing wrong data from column
df_new = df_new.reset_index(drop=True)    # Resetting Index


for i in range(0,len(df_new)):
  if(df_new.loc[i,'DCH / DCHC / CCC2 name'] == "Home Isolation"):    # Removing "Home Isolation" from 'DCH / DCHC / CCC2 name' and adding it in the column 'Action on Patient' of the same row
    df_new.loc[i,'DCH / DCHC / CCC2 name'] = "NA"       
    df_new.loc[i,'Action on Patient'] = "Home Isolation"
  if(df_new.loc[i,'Age'] in [0,0.0]):    # Removing people with age=0
    df_new = df_new.drop(index=i)

df_new = df_new.reset_index(drop=True)    # Resetting Index

# Cleaning the 'HealthPost Allocation' column
df_new = df_new.replace(["M", "F", "Death", "DEAD", "RECOVERED", "ACTIVE", "Discharge", "Kannamwar Nagar HP","PASSPOLI HP", "Paspoli Health Post", "Paspoli Village HP", "SONAPUR HEALTH POST", "SONAPUR HP", "Sonapur Health Post",\
                         "SUBHASH NAGAR HP", "Subhash Nagar Health Post", "BHANDUP MATERNITY HP", "Bhandup MH HP", "Bhandup Maternity Health Post", "HIRANANDANI HP", "Hiranandani Health Post", "ISHWAR NAGAR HP", "Ishwar Nagar Rch" \
                         "Kanjur Village Health Post", "Kanjur Village Hp", "KANJUR VILLAGE HP", "Kanjurmarg Rch 2", "KANJUR RCH", "KANNAMWAR NAGAR HP", "Kannamwar Health Post", "NEGATIVE REPORT", "Paspoli Health Post", "S", "S WARD",\
                         "S waRD", "Shivaji Talao Health Post", "SHIVAJI TALAO HP", "STHP", "TAEMBHIPADA HP", "TEMBHIPADA HP", "Tagore Nagar Health Post", "Tagore Nagar MH HP", "Tagore Nagar HP RCH2", "Tembhipada Health Post", \
                         "Tembhipada Hp", "Transferred", "Tulshet Pada Health Post", "Tulshetpada Hp", "TRANSFER", "NO", "no", "YES", "INDEX", "contact", "AT HOME", "At Home Worker", "At home", "At-home", "At-home worker", "HOME", "Home",\
                         "home", "Health worker", "OTHER", "Other essential services", "Other Essential Serveices", "Retire", "SLUM", "DEATH AT HOME", "Jumbo facility"], \
                        ["Male", "Female", "Dead", "Dead", "Recovered", "Active", "Recovered", "Kannamwar Health Post", "Passpoli Village HP", "Passpoli Village HP", "Passpoli Village HP", "Sonapur HP", "Sonapur HP", "Sonapur HP", \
                         "Subhash Nagar HP", "Subhash Nagar HP", "Bhandup Maternity HP","Bhandup Maternity HP","Bhandup Maternity HP", "Hiranandani HP","Hiranandani HP", "Ishwar Nagar HP", "Ishwar Nagar HP", "Kanjur HP", "Kanjur HP",\
                         "Kanjur HP" , "Kanjurmarg Rch-II HP", "Kanjur Rch", "Kannamwar HP", "Kannamwar HP", "","Passpoli Village HP", "S Ward", "S Ward", "S Ward", "Shivaji Talao HP", "Shivaji Talao HP", "Shivaji Talao HP"\
                         "Tembhipada HP", "Tembhipada HP", "Tagore Nagar HP", "Tagore Nagar HP", "Tagore Nagar Rch 2", "Tembhipada HP", "Tembhipada HP","", "Tulshetpada HP", "Tulshetpada HP", "", "No", "No", "Yes", "Index", "Contact", "At Home", \
                         "At Home", "At Home", "At Home", "At Home", "At Home", "At Home", "At Home", "Health Worker", "Other", "Other Essential Services", "Other Essential Services", "Retired", "Slum / Chawl", "Death At Home",\
                         "Jumbo Facility"])

df_new

In [ ]:
# For saving seperate sheets in one workbook
path = (r'/content/gdrive/MyDrive/IITB - INTERNSHIP/Dataset/Preprocessing_data/Cleaned_Dataset/Combined_linelist_data.xlsx')
df_new.to_excel(path, sheet_name = 'Combined_linelist')
wb = openpyxl.load_workbook(path)
writer = pd.ExcelWriter(path, engine = 'openpyxl')
writer.book = wb

## Some Analysis for Creating the Labelling functions

###1. *Creating Workbook with only Recovered Patient's details and worksheets with different analysis*


In [ ]:
df_recov = df_new.loc[df_new['Patient Status'] == 'Recovered'] # selecting the patients recovered
df_recov = df_recov.reset_index(drop=True)

recov_path = (r'/content/gdrive/MyDrive/IITB - INTERNSHIP/Dataset/Preprocessing_data/Cleaned_Dataset/Recovered_linelist_data.xlsx')
df_recov.to_excel(recov_path, sheet_name = 'Recovered_linelist')
wb_recov = openpyxl.load_workbook(recov_path)
writer_recov = pd.ExcelWriter(recov_path, engine = 'openpyxl')
writer_recov.book = wb_recov

In [ ]:
# Sorting according to Age
df_recov_age = df_recov.sort_values(by=['Age'])
df_recov_age.to_excel(writer_recov, sheet_name = 'Recovered_(sort-Age)_linelist')

# Sorting according to Gender
df_recov_gender = df_recov.sort_values(by=['Gender'])
df_recov_gender.to_excel(writer_recov, sheet_name = 'Recovered_(sort-gender)_linelist')

# Sorting according to Traceable
df_recov_traceable = df_recov.sort_values(by=['Traceable?'])
df_recov_traceable.to_excel(writer_recov, sheet_name = 'Recovered_(sort-tracable)_linelist')

# Sorting according to Out of Mumbai (Yes / No)
df_recov_mumbai = df_recov.sort_values(by=['Out of Mumbai (Yes / No)'])
df_recov_mumbai.to_excel(writer_recov, sheet_name = 'Recovered_(sort-Out_of_Mumbai)_linelist')

# Sorting according to Patient Type (Index / Contact)
df_recov_type = df_recov.sort_values(by=['Patient Type (Index / Contact)'])
df_recov_type.to_excel(writer_recov, sheet_name = 'Recovered_(sort-Patient_Type)_linelist')


### 2. Creating Workbook with only Dead Patient's details and worksheets with different analysis

In [ ]:
df_dead = df_new.loc[df_new['Patient Status'] == 'Dead'] # selecting the patients Dead
df_dead = df_dead.reset_index(drop=True)

dead_path = (r'/content/gdrive/MyDrive/IITB - INTERNSHIP/Dataset/Preprocessing_data/Cleaned_Dataset/Dead_linelist_data.xlsx')
df_dead.to_excel(dead_path, sheet_name = 'dead_linelist')
wb_dead = openpyxl.load_workbook(dead_path)
writer_dead = pd.ExcelWriter(dead_path, engine = 'openpyxl')
writer_dead.book = wb_dead

In [ ]:
# Sorting according to Age
df_dead_age = df_dead.sort_values(by=['Age'])
df_dead_age.to_excel(writer_dead, sheet_name = 'Dead_(sort-age)_linelist')

# Sorting according to Gender
df_dead_gender = df_dead.sort_values(by=['Gender'])
df_dead_gender.to_excel(writer_dead, sheet_name = 'Dead_(sort-gender)_linelist')

# Sorting according to Traceable
df_dead_traceable = df_dead.sort_values(by=['Traceable?'])
df_dead_traceable.to_excel(writer_dead, sheet_name = 'Dead_(sort-tracable)_linelist')

# Sorting according to Out of Mumbai (Yes / No)
df_dead_mumbai = df_dead.sort_values(by=['Out of Mumbai (Yes / No)'])
df_dead_mumbai.to_excel(writer_dead, sheet_name = 'Dead_(sort-Out_of_Mumbai)_linelist')

# Sorting according to Patient Type (Index / Contact)
df_dead_type = df_dead.sort_values(by=['Patient Type (Index / Contact)'])
df_dead_type.to_excel(writer_dead, sheet_name = 'Dead_(sort-Patient_Type)_linelist')

### 3. Creating Workbook with only Patient's details who are actively inffected with Covid-19 and worksheets with different analysis

In [ ]:
# Creating sheet with only Patient's details who are actively inffected with Covid-19

df_active = df_new.loc[df_new['Patient Status'] == 'Active'] # selecting the patients with Active case
df_active = df_active.reset_index(drop=True)

active_path = (r'/content/gdrive/MyDrive/IITB - INTERNSHIP/Dataset/Preprocessing_data/Cleaned_Dataset/Active_linelist_data.xlsx')
df_active.to_excel(active_path, sheet_name = 'Active_linelist')
wb_active = openpyxl.load_workbook(active_path)
writer_active = pd.ExcelWriter(active_path, engine = 'openpyxl')
writer_active.book = wb_active

In [ ]:
# Sorting according to Age
df_active_age = df_active.sort_values(by=['Age'])
df_active_age.to_excel(writer_active, sheet_name = 'Active_(sort-Age)_linelist')

# Sorting according to Gender
df_active_gender = df_active.sort_values(by=['Gender'])
df_active_gender.to_excel(writer_active, sheet_name = 'Active_(sort-gender)_linelist')

# Sorting according to Traceable
df_active_traceable = df_active.sort_values(by=['Traceable?'])
df_active_traceable.to_excel(writer_active, sheet_name = 'Active_(sort-tracable)_linelist')

# Sorting according to Out of Mumbai (Yes / No)
df_active_mumbai = df_active.sort_values(by=['Out of Mumbai (Yes / No)'])
df_active_mumbai.to_excel(writer_active, sheet_name = 'Active_(sort-Out_of_Mumbai)_linelist')

# Sorting according to Patient Type (Index / Contact)
df_active_type = df_active.sort_values(by=['Patient Type (Index / Contact)'])
df_active_type.to_excel(writer_active, sheet_name = 'Active_(sort-Patient_Type)_linelist')

In [ ]:
writer.save()
writer.close()

In [ ]:
writer_recov.save()
writer_recov.close()

In [ ]:
writer_dead.save()
writer_dead.close()

In [ ]:
writer_active.save()
writer_active.close()

### *Some Plottings*




In [ ]:
df_new['Age'].plot.box()
plt.title("Box Diagram - Distribution of Data according to Age")
plt.ylabel("Frequency")
plt.show()

In [ ]:
df_new.groupby('Gender')['Gender'].count().plot.pie(autopct='%.2f',figsize=(8,8),  subplots=True, labeldistance=None, legend=True, pctdistance = 0.9) # on complete dataset
plt.title("Distribution of data According to Gender")
plt.legend(loc="upper right")
plt.xlabel("")
plt.ylabel("")
plt.show()

In [ ]:
df_new.groupby('Patient Status')['Patient Status'].count().plot.pie(autopct='%.3f',figsize=(10,10),  subplots=True, labeldistance=None, legend=True, pctdistance = 0.9) # on recover dataset
plt.title("Distribution of Data According to Patient Status")
plt.xlabel("")
plt.ylabel("")
plt.legend(loc="upper right")
plt.show()

In [ ]:
df_active['Age'].plot.hist(label='Áctive', color='green', bins=50, alpha=0.5, figsize=(8,8))
plt.title("Histogram - Active cases analysis according to Age")
plt.xlabel("Age")
plt.ylabel("Frequency")
plt.legend(loc="upper right")
plt.show()

In [ ]:
df_dead['Age'].plot.hist(label='Dead', color='red', bins=50, alpha=0.5, figsize=(8,8))
plt.title("Histogram - Dead cases analysis according to Age")
plt.xlabel("Age")
plt.ylabel("Frequency")
plt.legend(loc="upper right")
plt.show()

In [ ]:
df_recov['Age'].plot.hist(label='Recovered', color='yellow', bins=50, alpha=0.3, figsize=(8,8))
plt.title("Histogram - Recovered cases analysis according to Age")
plt.xlabel("Age")
plt.ylabel("Frequency")
plt.legend(loc="upper right")
plt.show()

In [ ]:
df_active['Age'].plot.hist(label='Áctive',color='green', bins=50, alpha=0.5, figsize=(8,8))
df_dead['Age'].plot.hist(label='Dead',color='red', bins=50, alpha=0.5, figsize=(8,8))
df_recov['Age'].plot.hist(label='Recovered',color='yellow', bins=50, alpha=0.3, figsize=(8,8))
plt.title("Histogram - Dead, Active, Recovered cases analysis according to Age")
plt.xlabel("Age")
plt.ylabel("Frequency")
plt.legend(loc="upper right")
plt.show()

In [ ]:
df_dead.groupby('Is patient vaccinated')['Is patient vaccinated'].count().plot.pie(autopct='%.3f',figsize=(10,10), subplots=True, labeldistance=None, legend=True, pctdistance = 0.9) # on recover dataset
plt.title("Were patients vaccinated before Dying?")
plt.xlabel("")
plt.ylabel("")
plt.legend(loc="upper right")
plt.show()


In [ ]:
df_dead.groupby('2nd Dose')['2nd Dose'].count().plot.pie(autopct='%.2f',figsize=(10,10), subplots=True, labeldistance=None, legend=True, pctdistance = 0.9) # on recover dataset
plt.title("Dead Covid-19 Patients - 2nd Vaccine Dose?")

plt.axis('equal')
plt.xlabel("")
plt.ylabel("")
plt.legend(loc="upper right")
plt.show()

In [ ]:
df_dead.groupby('Out of Mumbai (Yes / No)')['Out of Mumbai (Yes / No)'].count().plot.pie(autopct='%.3f',figsize=(10,10), subplots=True, legend=True, pctdistance = 0.9) # on recover dataset
plt.title("Dead patients - Out of Mumbai?")
plt.axis('equal')
plt.xlabel("")
plt.ylabel("")
plt.legend(loc="upper right")
plt.show()

In [ ]:
df_dead.groupby('Select Comorbidities')['Select Comorbidities'].count().plot.pie(autopct='%.3f',figsize=(20,20),pctdistance=0.8) # on recover dataset
plt.title("Dead patients - Out of Mumbai?")
plt.xlabel("")
plt.ylabel("")
plt.legend(loc="upper right")
plt.show()


In [ ]:
df_dead.groupby('Patient Type (Index / Contact)')['Patient Type (Index / Contact)'].count().plot.pie(autopct='%.3f',figsize=(10,10), subplots=True, legend=True, pctdistance = 0.9) # on recover dataset
plt.title("Dead patients - Patient Type?")
plt.axis('equal')
plt.xlabel("")
plt.ylabel("")
plt.legend(loc="upper right")
plt.show()

In [ ]:
df_dead.groupby('Symptomatic?')['Symptomatic?'].count().plot.pie(autopct='%.3f',figsize=(10,10), subplots=True, legend=True, pctdistance = 0.9) # on recover dataset
plt.title("Dead patients - Symptomatic?")
plt.axis('equal')
plt.xlabel("")
plt.ylabel("")
plt.legend(loc="upper right")
plt.show()

In [ ]:
df_dead.groupby('Occupation')['Occupation'].count().plot.pie(autopct='%.3f',figsize=(40,40), subplots=True, legend=True, pctdistance = 0.9) # on recover dataset
plt.title("Dead patients - Occupation?")
plt.axis('equal')
plt.xlabel("")
plt.ylabel("")
plt.legend(loc="upper right")
plt.show()

In [ ]:
df_dead.groupby('If Patient is from Essential services, Private or Public Servic')['If Patient is from Essential services, Private or Public Servic'].count().plot.pie(autopct='%.3f',figsize=(10,10), subplots=True, legend=True, pctdistance = 0.9) # on recover dataset
plt.title("Dead patients - Essential Services?")
plt.axis('equal')
plt.xlabel("")
plt.ylabel("")
plt.legend(loc="upper right")
plt.show()

In [ ]:
df_dead.groupby('Patient is from SB (sealed building)')['Patient is from SB (sealed building)'].count().plot.pie(autopct='%.3f',figsize=(10,10), subplots=True, legend=True, pctdistance = 0.9) # on recover dataset
plt.title("Dead patients - Sealed Building?")
plt.axis('equal')
plt.xlabel("")
plt.ylabel("")
plt.legend(loc="upper right")
plt.show()

In [ ]:
df_dead.groupby('Patient is from CZ (containment zone)')['Patient is from CZ (containment zone)'].count().plot.pie(autopct='%.3f',figsize=(10,10), subplots=True, legend=True, pctdistance = 0.9) # on recover dataset
plt.title("Dead patients - Containment Zone?")
plt.axis('equal')
plt.xlabel("")
plt.ylabel("")
plt.legend(loc="upper right")
plt.show()

In [ ]:
df_new.groupby('Patient is from CZ (containment zone)')['Patient is from CZ (containment zone)'].count().plot.pie(autopct='%.3f',figsize=(10,10), subplots=True, legend=True, pctdistance = 0.9) # on recover dataset
plt.title("Dead patients - Containment Zone?")
plt.axis('equal')
plt.xlabel("")
plt.ylabel("")
plt.legend(loc="upper right")
plt.show()

In [ ]:
df_dead.groupby('Patient Location')['Patient Location'].count().plot.pie(autopct='%.3f',figsize=(10,10), subplots=True, legend=True, pctdistance = 0.9) # on recover dataset
plt.title("Dead patients - Containment Zone?")
plt.axis('equal')
plt.xlabel("")
plt.ylabel("")
plt.legend(loc="upper right")
plt.show()

In [ ]:
df_dead.groupby('Action on Patient')['Action on Patient'].count().plot.pie(autopct='%.3f',figsize=(10,10), subplots=True, legend=True, pctdistance = 0.9) # on recover dataset
plt.title("Dead patients - Action on Patients")
plt.axis('equal')
plt.xlabel("")
plt.ylabel("")
plt.legend(loc="upper right")
plt.show()

In [ ]:
df_dead['Total Number of High Risk Contacts'].plot.hist(label='High Risk Contact',color='red', bins=50, alpha=0.5, figsize=(8,8))
plt.title("Histogram - Dead Covid-19 Patients - High risk contacts")
plt.xlabel("High Risk Contact Counts")
plt.ylabel("Frequency")
plt.legend(loc="upper right")
plt.show()

In [ ]:
df_new.groupby('False Positive Case')['False Positive Case'].count().plot.pie(autopct='%.3f',figsize=(10,10), subplots=True, legend=True, pctdistance = 0.9) # on recover dataset
plt.title("False Positive Case")
plt.axis('equal')
plt.xlabel("")
plt.ylabel("")
plt.legend(loc="upper right")
plt.show()

In [ ]:
df_dead

In [ ]:
df_new.plot(x='Traceable?' ,y='Age',figsize=(50,5))

#**LABELING FUNCTIONS**

In [ ]:
import re
!git clone https://github.com/decile-team/spear.git
%cd spear

##REGULAR EXPRESSIONS

In [ ]:
Comorbidities = r'\bDiabetes Mellitus\b|\bHypertension\b|\bChronic Lung Diseases\b|\bChronic Kidney Disease\b|\bIschemic Heart Diseases\b|\bChronic Kidney Disease\b'

##LABELING FUNCTIONS
 

> 1 - Recover \
> 0 - Active \
> -1 - Death


### LFs - with Dictonary

In [ ]:
from spear.labeling import preprocessor

@preprocessor()
def create_dict(x):
    zip_iter = zip(col_names, x)
    return dict(zip_iter)

In [ ]:
import enum

class Labels(enum.Enum):
    recovered = 1
    active = 0
    dead = -1

In [ ]:
from spear.labeling import labeling_function, ABSTAIN

# Labeling Function 1 ---> Vaccinated and no comorbidity
@labeling_function(pre=[create_dict], label=Labels.recovered)
def lf_vaccinated(x,**kwargs):
  if (x['Is patient vaccinated']=="Yes" and x['Comorbidity']=="No"):    
      return Labels.recovered 
  return ABSTAIN


# Labeling Function 2 ---> Old Age, Comorbidity and symptomatic
@labeling_function(pre=[create_dict], label=Labels.dead)
def lf_old_symptomatic(x,**kwargs):
  if (x['Age']>60 and x['Comorbidity'] == "Yes" and x['Symptomatic?'] == "Yes"):    
      return Labels.dead
  return ABSTAIN


# Labeling Function 3 ---> symptomatic and from containment zone or sealed building
@labeling_function(pre=[create_dict], label=Labels.recovered)
def lf_symptomatic_cz_sb(x,**kwargs):
  if (x['Symptomatic?'] == "Yes" and (x['Patient is from CZ (containment zone)'] == "Yes" or x['Patient is from SB (sealed building)'] == "Yes")):    
      return Labels.recovered
  return ABSTAIN


# Labeling Function 4 ---> slum or chawl and comorbidity and symptomatic
@labeling_function(pre=[create_dict], label=Labels.dead)
def lf_slum_or_chawl_commorbidities(x,**kwargs):
  if ((x['Patient Location']=="Slum / Chawl" or x['Patient Location']=='Slum' or x['Patient Location']=='Chawl') and x['Comorbidity'] == "Yes" and x['Symptomatic?'] == "Yes"):
      return Labels.dead
  return ABSTAIN



# Labeling Function 5 ---> 'Action on Patient' is Home Isolation and Symptomatic= 'No'
@labeling_function(pre=[create_dict], label=Labels.recovered)
def lf_home_isolated(x,**kwargs):
  if (x['Action on Patient']=="Home Isolation" and x['Symptomatic?']=="No"):
      return Labels.recovered
  return ABSTAIN


# Labeling Function 6 ---> 
@labeling_function(pre=[create_dict], label=Labels.active)
def lf_containment_zone(x,**kwargs):
  if (x['Patient is from CZ (containment zone)'] == "Yes"):    
      return Labels.active
  return ABSTAIN


# Labeling Function 7 ---> 
@labeling_function(pre=[create_dict], label=Labels.active)
def lf_sealed_building(x,**kwargs):
  if (x['Patient is from SB (sealed building)'] == "Yes"):    
      return Labels.active
  return ABSTAIN


# Labeling Function 8 --->
@labeling_function(pre=[create_dict], label=Labels.dead)
def lf_commorbidities(x,**kwargs):
  if (x['Comorbidity'] == "Yes"):    
      return Labels.dead
  return ABSTAIN


# Labeling Function 9 --->
@labeling_function(pre=[create_dict], label=Labels.dead)
def lf_age(x,**kwargs):
  if (x['Age']>60):    
      return Labels.dead
  return ABSTAIN


# Labeling Function 10 --->
@labeling_function(pre=[create_dict], label=Labels.active)
def lf_symptomatic(x,**kwargs):
  if (x['Symptomatic?'] == "Yes"):
      return Labels.active
  return ABSTAIN
      

# Labeling Function 11 --->  
@labeling_function(pre=[create_dict], label=Labels.active)
def lf_essential_service(x,**kwargs):
  if (x['If Patient is from Essential services, Private or Public Servic'] == "Private"):
      return Labels.active
  return ABSTAIN


# Labeling Function 12 --->
@labeling_function(pre=[create_dict], label=Labels.active)
def lf_type(x,**kwargs):
  if (x['Patient Type (Index / Contact)'] == 'Index'):
      return Labels.active
  return ABSTAIN


# Labeling Function 13 --->
@labeling_function(pre=[create_dict], label=Labels.recovered)
def lf_home_building(x,**kwargs):
  if (x['Action on Patient']=='Home Isolation' and  x['Patient Location']=='Building'):
    return Labels.recovered
  return ABSTAIN


# Labeling Function 14 --->
@labeling_function(pre=[create_dict], label=Labels.active)
def lf_occupation(x,**kwargs):
  if (x['Occupation'] == 'Medical' or  x['Occupation']=='MP'):
    return Labels.active
  return ABSTAIN


# Labeling Function 15 --->
@labeling_function(pre=[create_dict], label=Labels.dead)
def lf_disease(x,**kwargs):
  if (re.search(Comorbidities, str(x['Select Comorbidities']), re.IGNORECASE)):
      return Labels.dead
  return ABSTAIN

### LFs - without dictonary

In [ ]:


# #Labeling function 1
# def lf_vaccinated(df):
#   for i in range(0,len(df)):
#     if (df.loc[i,'Is patient vaccinated'] == "Yes"):    
#       return 1
#     elif (df.loc[i,'Is patient vaccinated'] == "No"):    
#       return 0

# #Labeling function 2
# def lf_containment_zone(df):
#   for i in range(0,len(df)):
#     if (df.loc[i,'Patient is from CZ (containment zone)'] == "Yes"):    
#       return -1
#     elif (df.loc[i,'Patient is from CZ (containment zone)'] == "No"):    
#       return 0

# #Labeling function 3
# def lf_sealed_building(df):
#   for i in range(0,len(df)):
#     if (df.loc[i,'Patient is from SB (sealed building)'] == "Yes"):    
#       return -1
#     elif (df.loc[i,'Patient is from SB (sealed building)'] == "No"):    
#       return 0

# #Labeling function 4
# def lf_commorbidities(df):
#   for i in range(0,len(df)):
#     if (df.loc[i,'Comorbidity'] == "Yes"):    
#       return -1
#     elif (df.loc[i,'Comorbidity'] == "No"):    
#       return 0

# #Labeling function 5
# def lf_age(df):
#   for i in range(0,len(df)):
#     if (df.loc[i,'Age']>60 & df.loc[i,'Age']<80):    
#       return -1
#     elif (df.loc[i,'Age']<60):    
#       return 0

# #Labeling function 6
# def lf_disease(df):
#   for i in range(0,len(df)):
#     if (re.search(Comorbidities, str((df.loc[i,'Select Comorbidities']), re.IGNORECASE):    
#       return -1
#     else:    
#       return 0

# #Labeling function 7
# def lf_symptomatic(df):
#   for i in range(0,len(df)):
#     if (df.loc[i,'Symptomatic?']) == "Yes"):    
#       return -1
#     else:    
#       return 0
      

# #Labeling function 8    ---   Not sure of this function, but I made this based on the analysis (Pie diagram)
# def lf_essential_service(df):
#   for i in range(0,len(df)):
#     if (df.loc[i,'If Patient is from Essential services, Private or Public Servic']) == "Private"):    
#       return -1
#     else:    
#       return 0


# #Labeling function 9
# def lf_slum-chawl(df):
#   for i in range(0,len(df)):
#     if (df.loc[i,'Patient Location']) == "Slum / Chawl"):    
#       return -1
#     else:    
#       return 0

# #Labeling function 10
# def lf_type(df):
#   for i in range(0,len(df)):
#     if (df.loc[i,'Patient Type (Index / Contact)']) == "Index"):    
#       return -1
#     else:    
#       return 0

## LFset

In [ ]:
col_names=list(df_new.columns)
col_names

In [ ]:
X=df_new.to_numpy()
Y=df_new['Patient Status'].to_numpy()

In [ ]:
from spear.labeling import LFSet

LFS = [lf_vaccinated, lf_old_symptomatic, lf_symptomatic_cz_sb, lf_slum_or_chawl_commorbidities, lf_home_isolated, lf_containment_zone, lf_sealed_building, lf_commorbidities, lf_age, lf_symptomatic, lf_essential_service, lf_type, lf_home_building, lf_occupation, lf_disease]

rules = LFSet("SWARD_LFS")
rules.add_lf_list(LFS)

In [ ]:
from spear.labeling import PreLabels

sward_noisy_labels = PreLabels(name="sward",
                               data=X,
                               rules=rules,
                               labels_enum=Labels,
                               num_classes=3)

In [ ]:
sward_noisy_labels.generate_pickle("prelabels.pkl")

In [ ]:
sward_noisy_labels.generate_json('sward_json.json') #JSON

#**CAGE**

## Utility Function

In [ ]:
def get_test_U_data(X, Y, temp_len, test_size = 200, U_size = None):
    if U_size == None:
        U_size = len(X) - test_size
    index = np.arange(len(X))
    index = np.random.permutation(index)
    X = X[index]

    True_Labels = []
    
    for y in Y:
      if(y=="Recovered"):
        label=1
      elif(y=="Dead"):
        label=-1
      elif(y=="Active"):
        label=0
      True_Labels.append(label)
    
    Y = np.array(True_Labels)
    Y = Y[index]
    
    print(Y)

    X_T = X[-(test_size):]
    Y_T = Y[-(test_size):]
    R_T = np.zeros((test_size,temp_len))

    # X_U = X[:-(validation_size+test_size+L_size)]
    X_U = X[:U_size]
    # Y_U = Y[:-(validation_size+test_size+L_size)]
    R_U = np.zeros((U_size,temp_len))

    return X_T,Y_T,R_T, X_U,R_U

## Creating the unlabelled data and testing data

In [ ]:
test_size = 7000     # Size of Testing Data
U_size = 25000     # Size of unlabelled data
n_lfs = len(rules.get_lfs())    # Number of Labelling functions

X_T, Y_T, _, X_U, _= get_test_U_data(X, Y, n_lfs, test_size, U_size)

#print(X_T)    # Testing Data
print("Size of Testing Data:", len(X_T))   # Size of Testing Data
print("\n")

#print(Y_T)    # True Labels
print("Size of True Labels:", len(Y_T))   # Size of True Labels
print("\n")

#print(X_U)     # Unlabelled Data
print("Size of Unlabelled Data:", len(X_U))     # Size of Unlabelled Data

In [ ]:
path_json = '/content/gdrive/MyDrive/IITB - INTERNSHIP/Sward/CAGE/Data/sward_json.json'
T_path_pkl = '/content/gdrive/MyDrive/IITB - INTERNSHIP/Sward/CAGE/Data/sward_pickle_T.pkl' #test data - have true labels
U_path_pkl = '/content/gdrive/MyDrive/IITB - INTERNSHIP/Sward/CAGE/Data/sward_pickle_U.pkl' #unlabelled data - don't have true labels

log_path_cage_1 = '/content/gdrive/MyDrive/IITB - INTERNSHIP/Sward/Log/sward_log_1.txt' #cage is an algorithm, can be found below
params_path = '/content/gdrive/MyDrive/IITB - INTERNSHIP/Sward/Params/sward_params.pkl' #file path to store parameters of Cage, used below

In [ ]:
from spear.labeling import PreLabels

sward_noisy_labels = PreLabels(name="sward",
                               data=X_T,
                               gold_labels=Y_T,
                               rules=rules,
                               labels_enum=Labels,
                               num_classes=3)
sward_noisy_labels.generate_pickle(T_path_pkl)

sward_noisy_labels.generate_json(path_json) #generating json files once is enough

sward_noisy_labels = PreLabels(name="sward",
                               data=X_U,
                               rules=rules,
                               labels_enum=Labels,
                               num_classes=3) #note that we don't pass gold_labels here, for the unlabelled data

sward_noisy_labels.generate_pickle(U_path_pkl)

## Accessing Labeled Data

In [ ]:
from spear.utils import get_data, get_classes

data_U = get_data(path = U_path_pkl, check_shapes=True)
#check_shapes being True(above), asserts for relative shapes of arrays in pickle file
print("Number of elements in data list: ", len(data_U))
print("Shape of feature matrix: ", data_U[0].shape)
print("Shape of labels matrix: ", data_U[1].shape)
print("Shape of continuous scores matrix : ", data_U[6].shape)
print("Total number of classes: ", data_U[9])

classes = get_classes(path = path_json)
print("Classes dictionary in json file(modified to have integer keys): ", classes)

## Cage Algorithm

In [ ]:
from spear.cage import Cage

cage = Cage(path_json = path_json, n_lfs = n_lfs)

In [ ]:

cage = Cage(path_json = path_json, n_lfs = n_lfs)

probs = cage.fit_and_predict_proba(path_pkl = U_path_pkl, path_test = T_path_pkl, path_log = log_path_cage_1, \
                                   qt = 0.9, qc = 0.85, metric_avg = ['micro'], n_epochs = 200, lr = 0.001)
labels = np.argmax(probs, 1)
print("probs shape: ", probs.shape)
print("labels shape: ",labels.shape)

In [ ]:
cage = Cage(path_json = path_json, n_lfs = n_lfs)

labels = cage.fit_and_predict(path_pkl = U_path_pkl, path_test = T_path_pkl, path_log = log_path_cage_1, \
                              qt = 0.9, qc = 0.85, metric_avg = ['micro'], n_epochs = 200, lr = 0.01, \
                              need_strings = False)

print("labels shape: ", labels.shape)
print(type(labels[0]))

In [ ]:
cage = Cage(path_json = path_json, n_lfs = n_lfs)

labels_strings = cage.fit_and_predict(path_pkl = U_path_pkl, path_test = T_path_pkl, path_log = log_path_cage_1, \
                              qt = 0.9, qc = 0.85, metric_avg = ['micro'], n_epochs = 200, lr = 0.01, \
                              need_strings = True)

print("labels_strings shape: ", labels_strings.shape)
print(type(labels_strings[0]))

## Saving Parameters

In [ ]:
cage.save_params(save_path = params_path)

## Load Parameters

In [ ]:
cage_2 = Cage(path_json = path_json, n_lfs = n_lfs)
cage_2.load_params(load_path = params_path)

## Predict_proba function of Cage class

In [ ]:
probs_test = cage_2.predict_proba(path_test = T_path_pkl, qc = 0.85) 
print("probs_test shape: ",probs_test.shape)

## Predict function of Cage class

In [ ]:
labels_test = cage_2.predict(path_test = T_path_pkl, qc = 0.85, need_strings = False)
print("labels_test shape: ", labels_test.shape)

from sklearn.metrics import accuracy_score, f1_score

#Y_T is true labels of test data, type is numpy array of shape (num_instances,)
print("accuracy_score: ", accuracy_score(Y_T, labels_test))
print("f1_score: ", f1_score(Y_T, labels_test, average = 'micro'))